In [1]:
import pickle 
import pandas as pd
import spacy
from tqdm.auto import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm
tqdm.pandas()
import re
import transformers
from sentence_transformers import SentenceTransformer, util
import nltk
from nltk import tokenize
import pickle
import urllib.request as requests
import json
import statistics
import torch
from collections import defaultdict
import copy
import math
import os
import random
import numpy as np
import nltk
import pandas as pd
from sentence_transformers import InputExample, SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import torch
from torch.utils.data import DataLoader

2022-08-22 16:25:06.448116: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
pathtosave=os.getcwd()

pathtosave=pathtosave + "/models/SimCSE/"

In [3]:
model = SentenceTransformer(pathtosave)

df=pd.read_csv('gmo_to_uniqueId.csv')
materialID=df['lastRunJob'].to_list()

with open('New_PreProcessed_segmented.pkl', 'rb') as f:
    newmetalist=pickle.load(f)

sampled_datadf=pd.read_csv("Sampled_Adword_Labeled.csv")

#For Normal process with target labels only

df1=pd.read_csv('ChildAdwordTargetDatabase.csv')    
target_dict=dict(zip(df1['Name'], df1.index))

In [4]:
def clean(text):
    text=text.lower()
    text=text.replace("(", "")
    text=text.replace(")", "")
    text=text.replace(";", "")
    text=text.replace(",", "")
    text=text.replace("+", "")
    text=text.replace(".", "")
    text=text.replace("&", "")
    return text

In [ ]:
# def process_text(text):
#     #doc = self.nlp(unicode(text.lower()))
#     text=text.replace('\\n', ' ')
#     doc = nlp(text.lower())
#     result = []
#     for token in doc:
#         if token.text in nlp.Defaults.stop_words:
#             continue
#         if token.is_punct:
#             continue
#         if token.lemma_ == '-PRON-':
#             continue
#         result.append(token.lemma_)
#     return nlp(" ".join(result))

In [5]:
df1.head()

,Unnamed: 0,Name,info
0,0,Automotive,Automotive Crossover Hatchback Microcar Off-Ro...
1,1,Books and Literature,Books and Literature Art and Photography Books...
2,2,Business and Finance,Business and Finance Business Business Account...
3,3,Education,Education College Education Primary Education ...
4,4,Events and Attractions,Events and Attractions Fashion Events Malls & ...


In [6]:

# df1['Combined']=df1['Name'] #+' '+df1['info']
# df1['Combined1']=df1['Name'] +' '+df1['info']
# df1['Combined1'].apply(clean)
# df1['Combined1'].apply(process_text)
# df1['Combined'].apply(clean)

df1['info']=df1['info'].apply(clean)
#df1['info']=df1['info'].apply(process_text)
targetlist=df1['Name'].to_list()
# targetlistcalc=df1['Combined1'].to_list()
targetlistchild=df1['info']

In [7]:
print(targetlistchild)

0     automotive crossover hatchback microcar off-ro...
1     books and literature art and photography books...
2     business and finance business business account...
3     education college education primary education ...
4     events and attractions fashion events malls  s...
5     family and relationships dating parenting sing...
6     fine art costume dance design digital arts fin...
7     food  drink alcoholic beverages world cuisines...
8     healthy living fitness and exercise wellness c...
9     hobbies  interests musical instruments games a...
10    home  garden gardening remodeling  constructio...
11    medical health diseases and conditions allergi...
12    movies action and adventure movies romance mov...
13    music and audio adult contemporary music soft ...
14    pets birds cats dogs large animals reptiles fi...
15    pop culture celebrity deaths celebrity familie...
16    real estate apartments retail property hotel p...
17    religion  spirituality agnosticism spiritu

In [8]:
truthdatadf=pd.read_csv('AdwordLabeledDatabase.csv')
truthdatadf.head()

,Unnamed: 0.1,Unnamed: 0,materialId,lastRunJob,materialtitle,seriestitle,AssetCC,adword
0,3,3,GMO_00000000001313_01,FILE_MAF_20220211T205601Z_GMO_00000000001313_01,FULL HEARTS (EDITED),friday night lights,Is something happening\nbetween you and Riggin...,Sports
1,6,6,GMO_00000000001534_01,FILE_MAF_20220211T205607Z_GMO_00000000001534_01,MAY THE BEST MAN WIN (EDITED),friday night lights,BE HONEST WITH YOURSELF\nFOR TWO SECONDS AND T...,Sports
2,7,7,GMO_00000000001577_01,FILE_MAF_20210727T231031Z_GMO_00000000001577_01,GAVIN VOLURE,30 rock,"WE'RE AT A PARTY, LEMON,\nTRY TO LOOSEN UP A L...",Television
3,8,8,GMO_00000000001585_01,FILE_MAF_20210727T231030Z_GMO_00000000001585_01,DO-OVER,30 rock,"[bossa nova music] ♪ LLO, PUSSYCAT! (male voic...",Television
4,9,9,GMO_00000000001598_01,FILE_MAF_20210727T231031Z_GMO_00000000001598_01,CUTBACKS,30 rock,"[cheers] >> WHOO!\n>> WHOO! >> OKAY, GUYS, I J...",Television


In [9]:
def getadword(episode, truthdatadf):
    for i in range(len(truthdatadf.lastRunJob)):
        if episode == truthdatadf.lastRunJob[i]:
            return truthdatadf.adword[i]

In [ ]:
# temp=targetlistcalc[27]
# temp=temp.replace('*News business use case needs to be scoped by NBCU - priority TBD', '')
# targetlistcalc[27]=temp

In [10]:
sampled_datadf.head()

,Unnamed: 0.1,Unnamed: 0,materialId,lastRunJob,materialtitle,seriestitle,AssetCC,adword
0,665,665,GMO_00000000040741_01,FILE_MAF_20210731T020535Z_GMO_00000000040741_01,I KNEW YOU WHEN,friday night lights,"LKS, IS IS SLAMMIN' SAMMY MEADE,\nFO [radio]\n...",Sports
1,8244,8244,GMO_00000000370441_01,FILE_MAF_20220215T090159Z_GMO_00000000370441_01,Make Up or Break Up?,total bellas,>> NIKKI: Yeah! Welcome to Paris! Tonight on t...,Sports
2,678,678,GMO_00000000039091_01,FILE_MAF_20220213T121447Z_GMO_00000000039091_01,HOMECOMING (EDITED),friday night lights,"What's after\nhigh school, Brian? Me and Mack ...",Sports
3,777,777,GMO_00000000039094_01,FILE_MAF_20220213T173625Z_GMO_00000000039094_01,NEVERMIND (EDITED),friday night lights,"Here's to God in ten years from now, Street, g...",Sports
4,679,679,GMO_00000000039095_01,FILE_MAF_20220213T173619Z_GMO_00000000039095_01,WHAT TO DO WHILE YOU'RE WAITING (EDITED),friday night lights,"Dad? You're home! Hello, Brian. Waverly? Damn,...",Sports


In [11]:
listofepisodes=sampled_datadf['lastRunJob'].to_list()

In [ ]:
#listofepisodes=['FILE_MAF_20220211T205601Z_GMO_00000000001313_01', 'FILE_MAF_20220211T205607Z_GMO_00000000001534_01', 'FILE_MAF_20210727T231031Z_GMO_00000000001577_01' ]

In [12]:
def keydict(dataframe, targetlist):
    d=defaultdict(list)
    for target in targetlist:
        minidf=dataframe.loc[dataframe['adword']==target]
        minilist=minidf['lastRunJob'].to_list()
        d[target]=minilist
    
    return d



In [13]:
targetkey_filelist=keydict(sampled_datadf, targetlist)
print(targetkey_filelist)

defaultdict(<class 'list'>, {'Automotive': ['FILE_MAF_20211223T065128Z_GMO_00000000002584_01', 'FILE_MAF_20211223T064615Z_GMO_00000000106986_01', 'FILE_MAF_20211223T064819Z_GMO_00000000002709_01', 'FILE_MAF_20211222T231607Z_GMO_00000000294982_01', 'FILE_MAF_20211222T214634Z_GMO_00000000029298_01', 'FILE_MAF_20211222T214941Z_GMO_00000000121162_01', 'FILE_MAF_20211222T230805Z_GMO_00000000273733_01', 'FILE_MAF_20211223T065125Z_GMO_00000000027759_01', 'FILE_MAF_20211223T065130Z_GMO_00000000002708_01', 'FILE_MAF_20211223T063010Z_GMO_00000000002843_01', 'FILE_MAF_20211223T064314Z_GMO_00000000029163_01', 'FILE_MAF_20211222T214633Z_GMO_00000000027597_01', 'FILE_MAF_20211223T065127Z_GMO_00000000002637_01', 'FILE_MAF_20211223T065129Z_GMO_00000000009448_01', 'FILE_MAF_20211223T064919Z_GMO_00000000002707_01', 'FILE_MAF_20211222T214633Z_GMO_00000000027055_01', 'FILE_MAF_20211223T065126Z_GMO_00000000002685_01', 'FILE_MAF_20211223T065020Z_GMO_00000000009009_01', 'FILE_MAF_20211223T061005Z_GMO_0000000

In [ ]:
# #NRS bulk with top 1 save to file regular
# dict_to_save=defaultdict(list)
# for target in tqdm(targetlist):
#     scores=[]
#     for ep in tqdm(listofepisodes):
#         index=materialID.index(ep)
#         groundtruth=getadword(ep,truthdatadf)
#         # if groundtruth in target_dict:
#         #         x=target_dict[groundtruth]
#         x=target_dict[target]
#         #print(groundtruth)
#         print(x)
#         segmentscore=[]
#         for i in range(len(newmetalist[index])):
#             if len(newmetalist[index][i])==0:
#                 continue
#             emb1=model.encode(newmetalist[index][i]) #, convert_to_tensor=True)
#             emb2=model.encode(targetlist) #, convert_to_tensor=True)
#             cos_sim=util.cos_sim(emb1,emb2)

#             #For OG Class
#             MaxAggregation = {}
#             for i in range(cos_sim.shape[1]):
#                 all_sentence_combinations=[]
#                 for j in range(cos_sim.shape[0]):
#                     all_sentence_combinations.append([cos_sim[j][i], j, i])
#                 all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)
#                 MaxAggregation[i]=all_sentence_combinations[0]
                

#             segmentscore.append(MaxAggregation[x][0])

            
                    
#         avg=(sum(segmentscore)/len(segmentscore))
#         scores.append(avg)
        
#     print(scores)
#     dict_to_save[target]=scores

In [ ]:
# with open('NRS_top1_regular.pkl', 'wb') as f:
#     pickle.dump(dict_to_save, f)


In [ ]:
# with open('NRS_top1_regular.pkl', 'rb') as f:
#     NRS_dict=pickle.load(f)


In [14]:
def get_key(val):
    for key, value in target_dict.items():
         if val == value:
             return key
 
    return "key doesn't exist"

In [15]:
#Relevance calculation per episode top1
dictofrelevance=copy.deepcopy(targetkey_filelist)
for key, value in tqdm(targetkey_filelist.items()): #for each IAB Label
    if len(targetkey_filelist[key])==0:
        continue
    for i in tqdm(range(len(targetkey_filelist[key]))): #for each File
        index=materialID.index(targetkey_filelist[key][i])
        x=target_dict[key] #Index of the target label
        MaxAggregation=defaultdict(list)

        for l in range(len(newmetalist[index])): #For each Segment
            # print(len(newmetalist[index]))
            if len(newmetalist[index][l])==0: 
                continue
            emb1=model.encode(newmetalist[index][l]) #, convert_to_tensor=True)
            emb2=model.encode(targetlistchild) #, convert_to_tensor=True)
            cos_sim=util.cos_sim(emb1,emb2)

        #Add all pairs to a list with their cosine similarity score
        
            for k in range(cos_sim.shape[1]):
                all_sentence_combinations=[]
                for j in range(cos_sim.shape[0]):
                    all_sentence_combinations.append([cos_sim[j][k], j, k])
                all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)
                MaxAggregation[k].append(all_sentence_combinations[0])
    
     
        dictofrelevance[key][i]={dictofrelevance[key][i]:MaxAggregation}
     

with open('Relevance_SimCSE.pkl', 'wb') as f:
    pickle.dump(dictofrelevance, f)   
 
        #RS and NRS for OG Class

        # RSlist=MaxAggregation[x]
        # score=[]
        # for s in RSlist:
        #     score.append(s[0])

        # rs=sum(score)/len(score)

        # NRSlist=NRS_dict[key]
        # nrsdenom=sum(NRSlist)-rs/len(NRSlist)

        # NRS=rs/nrsdenom

        
        # #RS and ClassPredicted for Predicted Class

        # order=[]
        # for key in MaxAggregation.keys():
        #     l=MaxAggregation[key]
        #     summ=[]
        #     for val in l:
        #         summ.append(val[0])
        #     order.append(sum(summ))
        
        
        # max_val=max(order)
        # max_index=order.index(max_val)
        # ClassPredicted=get_key(max_index)

        # RSlistss=MaxAggregation[max_index]
        # scoress=[]
        # for s in RSlistss:
        #     scoress.append(s[0])

        # RS_CP=sum(scoress)/len(scoress)

        # d={targetkey_filelist[key][i]:{'NRS': NRS, "Class_Predicted": ClassPredicted, "RS_CP": RS_CP, "RS_OG": rs }}

        


        


        

    

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [16]:
with open('Relevance_SimCSE.pkl', 'rb') as f:
     Relevance_dict=pickle.load(f)

In [17]:
#Def Top1 Agg
def segmentcombiner(dic): #for segments to final dic of relevance scores
    dic1=copy.deepcopy(dic)
    filename=list(dic1.keys())[0]
    defdict=dic1[filename]
    for k,v in defdict.items():
        RSlist=defdict[k]
        score=[]
        for s in RSlist:
            score.append(s[0])

        rs=sum(score)/len(score)
        defdict[k]=rs
    d={filename:defdict}
    return d
    


In [ ]:
#Def Top3 and Harm Agg
# def segmentcombinertop3Harm(dic): #for segments to final dic of relevance scores
#     dic1=copy.deepcopy(dic)
#     filename=list(dic1.keys())[0]
#     defdict=dic1[filename]
#     for k,v in defdict.items():
#         RSlist=defdict[k][0]
#         score=[]
#         for s in RSlist:
#             score.append(s[0])

#         rs=sum(score)/len(score)
#         defdict[k]=rs
#     d={filename:defdict}
#     return d

In [ ]:
# with open('Relevance_regular.pkl', 'rb') as f:
#      Relevance_dict=pickle.load(f)

In [19]:
print(NRS(Relevance_dict))

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

...

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

{'Automotive': 1.1038001837779903, 'Books and Literature': 0.9926491622200283, 'Business and Finance': 1.0290048029361425, 'Education': 1.154467638061618, 'Events and Attractions': 1.05181891492986, 'Family and Relationships': 1.0694728664272874, 'Food & Drink': 1.3894025419101697, 'Hobbies & Interests': 1.0105054261463595, 'Medical Health': 1.2543368182165826, 'Music and Audio': 1.0292899132398017, 'Pets': 1.0265670032065435, 'Real Estate': 1.5735022311513598, 'Religion & Spirituality': 1.0518353315076476, 'Science': 0.9281737516857087, 'Shopping': 0.9714797868249403, 'Sports': 1.0712163236442018, 'Style & Fashion': 1.4987482757123105, 'Technology & Computing': 1.3425803722028316, 'Television': 1.0144864459966496, 'Travel': 1.1898603374097056}
None


In [18]:
#Def NRS for top1 
def NRS(dict):
    d={}
    for k,v in tqdm(dict.items()):#for each label
        if len(dict[k]) == 0:
            continue
        d1={}
        for target in tqdm(targetlist): #for each topic
            scoreforfiles=[]
            if len(dict[target]) == 0:
                continue
            for i in range(len(dict[k])): #for each file in label
                x=target_dict[target]
                combinedsegments=segmentcombiner(dict[k][i])
                key=list(combinedsegments.keys())[0]
                scoreforfiles.append(float(combinedsegments[key][x])) #combine segments 
            avg=sum(scoreforfiles)/len(scoreforfiles)
            d1[target]=avg
        d[k]=d1

    finaldict={}
    for k,v in d.items():
        numerator=d[k][k]
        denominator=[]
        for kk,vv in d.items():
            denominator.append(d[kk][k])
        avgdenominator=(sum(denominator)-numerator)/(len(denominator)-1)
        NRS=numerator/avgdenominator
        finaldict[k]=NRS
    print(finaldict)


            
        

In [ ]:
print(RSMetrics(Relevance_dict))

In [ ]:
#Def RS at 0.15, 0.20, 0.35 and Mean RS 
def RSMetrics(dict):
    d={}
    for k,v in tqdm(dict.items()): #for each label
        if len(dict[k]) == 0:
            continue
        scoreforfiles=[]
        for i in range(len(dict[k])): #for each file in label
            x=target_dict[k]
            combinedsegments=segmentcombiner(dict[k][i])
            key=list(combinedsegments.keys())[0]
            scoreforfiles.append(round(float(combinedsegments[key][x]),2))
        rs15=[]
        rs20=[]
        rs35=[]
        for item in scoreforfiles:
            if item >= 0.15:
                rs15.append(item)
            if item >= 0.20:
                rs20.append(item)
            if item >= 0.35:
                rs35.append(item)

        RS15=(len(rs15)/len(scoreforfiles))*100
        RS20=(len(rs20)/len(scoreforfiles))*100
        RS35=(len(rs35)/len(scoreforfiles))*100
        d1={"RS15":RS15, "RS20": RS20, "RS35": RS35}
        d[k]=d1
    
    return d
        
    


In [ ]:
#Def RS at 0.15, 0.20, 0.35 and Mean RS 
def RSMetricsHM(dict):
    d={}
    for k,v in tqdm(dict.items()): #for each label
        if len(dict[k]) == 0:
            continue
        scoreforfiles=[]
        for i in range(len(dict[k])): #for each file in label
            x=target_dict[k]
            combinedsegments=segmentcombiner(dict[k][i])
            key=list(combinedsegments.keys())[0]
            ll=(dict[k][i][key][x])
            lll=[]
            for score in ll:
                print(score[1])
                lll.append(score[1][0][2])
            avg=sum(lll)/len(ll)
            scoreforfiles.append(round(float(avg),2))
        rs15=[]
        rs20=[]
        rs35=[]
        for item in scoreforfiles:
            if item >= 0.15:
                rs15.append(item)
            if item >= 0.20:
                rs20.append(item)
            if item >= 0.35:
                rs35.append(item)

        RS15=(len(rs15)/len(scoreforfiles))*100
        RS20=(len(rs20)/len(scoreforfiles))*100
        RS35=(len(rs35)/len(scoreforfiles))*100
        d1={"RS15":RS15, "RS20": RS20, "RS35": RS35}
        d[k]=d1
    
    return d
        

In [ ]:
print(Prediction(Relevance_dict))

In [20]:
#Def Predicted Class and RS of Class
def Prediction(dict):
    d={}
    for k,v in tqdm(dict.items()): #for each label
        if len(dict[k]) == 0:
            continue
        d1={}
        for i in range(len(dict[k])): #for each file in label
            combinedsegments=segmentcombiner(dict[k][i])
            key=list(combinedsegments.keys())[0]
            order=[]
            for kk in combinedsegments[key].keys():
                l=combinedsegments[key][kk]
                order.append(l)
            max_val=max(order)
            max_index=order.index(max_val)
            ClassPredicted=get_key(max_index)
            RS_CP=combinedsegments[key][max_index]
            d2={'ClassPredicted': ClassPredicted, 'RS_CP': RS_CP }
            d1[key]=d2
        d[k]=d1

    return d

            # 
            # scoreforfiles.append(float(combinedsegments[key][x]0))

            # order=[]
            # for key in MaxAggregation.keys():
            #     l=MaxAggregation[key]
            #     summ=[]
            #     for val in l:
            #         summ.append(val[0])
            #     order.append(sum(summ))
            
            
            # max_val=max(order)
            # max_index=order.index(max_val)
            # ClassPredicted=get_key(max_index)

            # RSlistss=MaxAggregation[max_index]
            # scoress=[]
            # for s in RSlistss:
            #     scoress.append(s[0])

            # RS_CP=sum(scoress)/len(scoress)
    

In [22]:
print(mAP(Relevance_dict))

  0%|          | 0/29 [00:00<?, ?it/s]

({'Automotive': 0.0, 'Books and Literature': 5.0, 'Business and Finance': 15.0, 'Education': 20.0, 'Events and Attractions': 0.0, 'Family and Relationships': 45.0, 'Food & Drink': 65.0, 'Hobbies & Interests': 0.0, 'Medical Health': 60.0, 'Music and Audio': 25.0, 'Pets': 15.0, 'Real Estate': 90.0, 'Religion & Spirituality': 15.0, 'Science': 0.0, 'Shopping': 5.0, 'Sports': 15.0, 'Style & Fashion': 100.0, 'Technology & Computing': 0.0, 'Television': 0.0, 'Travel': 5.0}, 24.0)


In [21]:
#Def mAP
def mAP(dic):
    res=Prediction(dic)
    d={}
    mAP=[]
    for k,v in res.items():
        classes=[]
        truth=[]

        for kk,vv in res[k].items():
            ClassPred=res[k][kk]['ClassPredicted']
            if ClassPred==k:
                classes.append(ClassPred)
            truth.append(k)
        precision=(len(classes)/len(truth))*100
        mAP.append(precision)
        d[k]=precision
    mAPfinal=sum(mAP)/len(mAP)
    return(d,mAPfinal)
  




In [ ]:
Relevance_dict.items()

In [78]:
def topksentences(dict):
    d=defaultdict(list)
    for k,v in tqdm(dict.items()):#for each label
        if len(dict[k]) == 0:
            continue
        d1={}
        scoreforfiles=[]
 
        for i in range(len(dict[k])): #for each file in label
            x=target_dict[k]
            seg_rel=copy.deepcopy(dict[k][i])
            filename=list(seg_rel.keys())[0]
            defdict=seg_rel[filename]
            segments=defdict[x]
            for count,seg in enumerate(segments):
                seg.append(filename)
                seg.append(count)                
                scoreforfiles.append(seg)
        sort=sorted(scoreforfiles, key=lambda x: x[0], reverse=True)
        Final=sort[0:20]
        
        for items in Final:
            index=materialID.index(items[3])
            try:
                items.append(newmetalist[index][items[4]][items[1]])
            except Exception as e:
                print(e)
                print(index,items[4],items[1])
                continue
        print(Final)
        ff=[]
        for f in Final:
            ff.append(f[5])
    
        d[k].append(ff)
    
        

                
                

In [58]:
index=materialID.index('FILE_MAF_20211223T065129Z_GMO_00000000009448_01')
print(index)

5659


In [79]:
topksentences(Relevance_dict)

  0%|          | 0/29 [00:00<?, ?it/s]

[[tensor(0.4367), 420, 0, 'FILE_MAF_20211223T065129Z_GMO_00000000009448_01', 1, 'the most successful new segment of the food service industry today food truck .'], [tensor(0.3451), 91, 0, 'FILE_MAF_20211223T061005Z_GMO_00000000113301_01', 1, 'it will have a rental tag in the front windshield and one of those license plate border .'], [tensor(0.3004), 40, 0, 'FILE_MAF_20211223T064314Z_GMO_00000000029163_01', 1, 'alarm sound   truck 81 squad 3 ambulance 61 . single vehicle accident .'], [tensor(0.2938), 423, 0, 'FILE_MAF_20211223T064819Z_GMO_00000000002709_01', 0, 'this be one of they .'], [tensor(0.2903), 63, 0, 'FILE_MAF_20211222T231607Z_GMO_00000000294982_01', 4, 'for example it feature an ergonomic nonslip grip and it be make from a lightweight alloy develop for formula 1 racing .'], [tensor(0.2886), 42, 0, 'FILE_MAF_20211222T174857Z_GMO_00000000334049_01', 0, 'alarm blare   truck 81 ambulance 61 . car accident .'], [tensor(0.2794), 378, 0, 'FILE_MAF_20211223T064919Z_GMO_000000000027